In [3]:
import os
def find_image_file(base_path, file_name_without_extension):
    # Define the possible image extensions
    image_extensions = ['.jpg', '.png', '.JPG', '.tif']
    
    # Search for image files with the specified base name and extensions
    for extension in image_extensions:
        # Build the path for the image file
        image_path = os.path.join(base_path, file_name_without_extension + extension)
        if os.path.exists(image_path):
            return image_path  # Return the image path if file exists

    return None  # If no image file is found

In [4]:
import os
from pathlib import Path
import cv2
import json
from tqdm import tqdm

# Directory paths
json_dir = "/trunk2/yufei/datasets/jsons/test"
image_dir = "/trunk2/yufei/datasets/pictures/test"
output_dir = "/trunk2/yufei/arabic_ms_data_to_teklia/teklia/ground_true"

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# List all JSON files in the directory
json_files = list(Path(json_dir).rglob('*.json'))

# Helper function to find corresponding image file
def find_image_file(base_path, file_name_without_extension):
    image_extensions = ['.jpg', '.png', '.JPG', '.tif']
    for extension in image_extensions:
        image_path = os.path.join(base_path, file_name_without_extension + extension)
        if os.path.exists(image_path):
            return image_path
    return None

# Process each JSON file
for json_file_path in tqdm(json_files, desc='Processing JSON files'):
    with open(str(json_file_path), 'r') as f:
        annotations_data = json.load(f)

    record = {'img_size': [], 'text_line': []}
    file_name_without_extension = json_file_path.stem
    image_file = find_image_file(image_dir, file_name_without_extension)

    if image_file is None:
        print(f"No image found for {json_file_path.name}, skipping.")
        continue

    image = cv2.imread(image_file)
    height, width = image.shape[:2]
    # Swap the width and height for the 'img_size'
    record['img_size'] = [height, width]

    for annotation in annotations_data:
        if 'polygon' in annotation and 'confidence' in annotation:
            # Swap the x and y coordinates for each point in the polygon
            swapped_polygon = [[point[1], point[0]] for point in annotation['polygon']]
            text_line_entry = {
                'confidence': annotation['confidence'],
                # Ensure the polygon is closed by repeating the first point
                'polygon': swapped_polygon + [swapped_polygon[0]] if swapped_polygon else []
            }
            record['text_line'].append(text_line_entry)

    output_path = os.path.join(output_dir, file_name_without_extension + '.json')
    with open(output_path, 'w') as outfile:
        json.dump(record, outfile, indent=4)

print("All files processed.")


Processing JSON files: 100%|██████████| 45/45 [00:09<00:00,  4.81it/s]

All files processed.
